In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import math

In [3]:
df = pd.read_csv("~/Data/clean_rating4.csv").drop(["id"], axis = 1)
train_ratio = 0.9
train_data = df[:int(len(df) * train_ratio)]
test_data = df[int(len(df) * train_ratio):]
i2u = defaultdict(list)
for data in train_data.values:
    i2u[data[1]].append(data[0])

In [4]:
def ndcg(k, ranklist, testlist):
    if not testlist: return 0
    idcg_k, dcg_k = 0, 0
    if len(testlist) < k:
        k = len(testlist)
    for i in range(k):
        idcg_k += 1 / math.log(i + 2, 2)
        
    s = set(testlist)
    hits = [idx for idx, val in enumerate(ranklist) if val in s]
    count = len(hits)
    
    for i in range(count):
        dcg_k += 1 / math.log(hits[i] + 2, 2)
        
    return float(dcg_k / idcg_k)

In [5]:
test_users, simi, precision, final = set(test_data.user_id.tolist()), {}, [], []
recall, f1, ndcg_k = [], [], []
for k in range(10):
    ndcg_k = []
    precision, simi = [], {}
    for user in test_users:
        animes = test_data[test_data.user_id == user]
        items = animes[animes.rating == 10].anime_id.tolist()
        if items: 
            item = items[0]
        else: 
            for i in range(9, -1, -1):
                items = animes[animes.rating == i].anime_id.tolist()
                if items:
                    item = items[0]
                    break
                    
        for key in i2u.keys():
            simi[(item, key)] = len(set(i2u[item]) & set(i2u[key])) / math.sqrt(len(i2u[item]) * len(i2u[key]))

        rec = [item[0][1] for item in sorted(simi.items(), key = lambda x: x[1], reverse = True)][1: 6]
        
        temp = animes[animes.rating > k].anime_id.tolist()
#         both = set(temp) & set(rec)
#         precise = len(both) / len(rec)
        
#         precision.append(precise)
#         recall.append(len(both) / len(temp))
        ndcg_k.append(ndcg(10, rec, temp))

    final.append(np.mean(ndcg_k))

0
1
2
3
4
5
6
7
8
9


In [6]:
print(final)

[0.1891258420383167, 0.1891258420383167, 0.1890568548735645, 0.18891888054406006, 0.18830117345271757, 0.18685639036066637, 0.18416961370394053, 0.17889090243610026, 0.169930179368218, 0.15828371822675852]


In [17]:
m, similarity, i2u = len(item_to_user), {}, list(item_to_user.items())
for i in range(m):
    if not i % 100: print(i)
    for j in range(i, m):
        similarity[(i2u[i][0], i2u[j][0])] = len(set(i2u[i][1]) & set(i2u[j][1])) / math.sqrt(len(i2u[i][1]) * len(i2u[j][1]))

print(len(similarity))

0
100


KeyboardInterrupt: 